# Regmap Parsing

In [1]:
import ipq_pynq_utils
import numpy as np

In [2]:
clk104 = ipq_pynq_utils.CLK104("clockfile.txt")

In [3]:
clk104.PLL2_FREQ = 2457.6 # Main Reference Clock
# clk104.PLL2_FREQ = 3000 # Main Reference Clock

print(f"ADC_REFCLK: {clk104.ADC_REFCLK.freq:.2f}")
print(f"DAC_REFCLK: {clk104.DAC_REFCLK.freq:.2f}")
print(f"RF_PLL_ADC_REF: {clk104.RF_PLL_ADC_REF.freq:.2f}")
print(f"RF_PLL_DAC_REF: {clk104.RF_PLL_DAC_REF.freq:.2f}")
print(f"SYSREF_FREQ: {clk104.SYSREF_FREQ:.2f}")

ADC_REFCLK: 245.76
DAC_REFCLK: 245.76
RF_PLL_ADC_REF: 245.76
RF_PLL_DAC_REF: 245.76
SYSREF_FREQ: 7.68


In [4]:
for i,branch in enumerate(clk104.lmk.clock_branches):
    print(i, branch.dclk_active, branch.sdclk_active)

0 True True
1 False True
2 True True
3 False False
4 True True
5 False True
6 False False


In [5]:
# clk104.get_register_dump()

In [6]:
clk104.lmk.update(printDebug=False)

In [7]:
clk104.lmk.write_register_dump("clockfile_out.txt")

In [8]:
!diff clockfile.txt clockfile_out.txt

136c136
< R8191	0x1FFF53
\ No newline at end of file
---
> R8191	0x1FFF53


In [9]:
len(bin(0x1fff))-2

13

In [10]:
def parse_register_file(filename, aw=13, dw=8):
    with open(filename, "r") as f:
        lines = f.read().strip().split("\n")

    ret = []

    for line in lines:
        a,b = line.split("\t")
        data = int(b, 16)
        mask = (1 << aw)-1
        data_mask = (1 << dw) - 1

        ret.append((mask & (data >> dw), data & data_mask))

    return ret

In [11]:
prog_file = parse_register_file("clockfile.txt")

In [12]:
print([a[0] for a in prog_file])

[0, 0, 2, 3, 4, 5, 6, 12, 13, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 369, 370, 380, 381, 358, 359, 360, 361, 362, 363, 364, 365, 366, 371, 386, 387, 388, 389, 392, 393, 394, 395, 8189, 8190, 8191]


In [13]:
if 0:
    for addr,data in prog_file:
        # print(f"Explaining register 0x{addr:x} = 0x{data:x}")
        if not addr in register_by_addr:
            print(f"Unhandled register: {hex(addr)}, skipping ...")
            print()
            continue

        register = register_by_addr[addr]
        print(hex(addr))
        register.parse(data)

        for field in register.fields:
            val = field.get()
            if field.name == "CONST":
                continue

            print(f"    {field.name}")
            print(f"        Description: {field.description}")
            print(f"        Value:       {val}")
            if field.valid_type == "enum":
                print(f"        ValDesc:     {field.value_description}")
            print()

In [14]:
lmx_clock_files = [
    "LMX2594_REF-245M76__OUT-0204M80_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-0245M76_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-0491M52_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-1024M00_11052019_D.txt",
    "LMX2594_REF-245M76__OUT-1966M08_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-2211M84_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-2457M60_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-3072M00_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-3932M16_10172019_I.txt",
    "LMX2594_REF-245M76__OUT-4096M00_11052019_D.txt",
    "LMX2594_REF-245M76__OUT-4423M68_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-4915M20_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-5898M24_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-6144M00_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-6881M28_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-7372M80_10172019_I_D.txt",
    "LMX2594_REF-245M76__OUT-7864M32_10172019_I.txt",
    "LMX2594_REF-245M76__OUT-8192M00_11052019.txt",
    "LMX2594_REF-245M76__OUT-8847M36_10172019_I.txt",
    "LMX2594_REF-245M76__OUT-9830M40_10172019_I.txt"
]

In [15]:
lmx = ipq_pynq_utils.LMX2594(245.76)

In [16]:
for f in lmx_clock_files:
    lmx.init_from_file(f"../ipq_pynq_utils/data/clockFiles/{f}")
    print(f"f_outa = {round(lmx.f_outa, ndigits=2):7.2f}, f_vco = {round(lmx.f_vco, ndigits=2):8.2f}")# , f_outb = {round(lmx.f_outb, ndigits=3)}")

f_outa =  204.80, f_vco =  9830.40
f_outa =  245.76, f_vco =  7864.32
f_outa =  491.52, f_vco =  7864.32
f_outa = 1024.00, f_vco =  8192.00
f_outa = 1966.08, f_vco =  7864.32
f_outa = 2211.84, f_vco =  8847.36
f_outa = 2457.60, f_vco =  9830.40
f_outa = 3072.00, f_vco = 12288.00
f_outa = 3932.16, f_vco =  7864.32
f_outa = 4096.00, f_vco =  8192.00
f_outa = 4423.68, f_vco =  8847.36
f_outa = 4915.20, f_vco =  9830.40
f_outa = 5898.24, f_vco = 11796.48
f_outa = 6144.00, f_vco = 12288.00
f_outa = 6881.28, f_vco = 13762.56
f_outa = 7372.80, f_vco = 14745.60
f_outa = 7864.32, f_vco =  7864.32
f_outa = 8192.00, f_vco =  8192.00
f_outa = 8847.36, f_vco =  8847.36
f_outa = 9830.40, f_vco =  9830.40


In [17]:
num = lmx.get_long_register(lmx.PLL_NUM_31_16, lmx.PLL_NUM_15_0)
den = lmx.get_long_register(lmx.PLL_DEN_31_16, lmx.PLL_DEN_15_0)
pll_n = lmx.get_long_register(lmx.PLL_N_18_16, lmx.PLL_N_15_0)

In [18]:
lmx.f_outa

9830.4

In [19]:
f_pd = 24.576
f_vco = f_pd * (pll_n + num/den)

In [20]:
f_vco

9830.4

In [21]:
lmx.OUTA_MUX.get()

VCO

In [22]:
lmx.SYSREF_DIV.get()

1

In [23]:
f_out = 4915.2

In [24]:
lmx.init_from_file("../ipq_pynq_utils/data/clockFiles/LMX2594_REF-245M76__OUT-3072M00_10172019_I_D.txt")

In [45]:
lmx.set_output_frequency(1966.08)

print("2G")

lmx.set_output_frequency(1966.08)

Solutions:
  i    f_vco   DIV MIN_N DLY_SEL    n    R   R_pre f_fpd Metric
----------------------------------------------------------------
  0: 7864.32 /   4    28       1   32    1     1  245.76 128
  1: 11796.48 /   6    28       1   48    1     1  245.76 288
Choosing solution 0 with minimal metric 128
2G
Solutions:
  i    f_vco   DIV MIN_N DLY_SEL    n    R   R_pre f_fpd Metric
----------------------------------------------------------------
  0: 7864.32 /   4    28       1   32    1     1  245.76 128
  1: 11796.48 /   6    28       1   48    1     1  245.76 288
Choosing solution 0 with minimal metric 128


In [44]:
lmx.f_outa

2000.299746835443

In [47]:
1966.08 / 245.76

8.0

In [27]:
lmx.init_from_file("../ipq_pynq_utils/data/clockFiles/LMX2594_REF-245M76__OUT-3072M00_10172019_I_D.txt")
lmx.set_output_frequency(f_out)
a = np.array(lmx.get_register_dump())

print(lmx.OUTA_PWR.value)

Solutions:
  i    f_vco   DIV MIN_N DLY_SEL    n    R   R_pre f_fpd Metric
----------------------------------------------------------------
  0: 9830.40 /   2    28       1   40    1     1  245.76 80
Choosing solution 0 with minimal metric 80
31


In [28]:
lmx.init_from_file("../ipq_pynq_utils/data/clockFiles/LMX2594_REF-245M76__OUT-1024M00_11052019_D.txt")
lmx.set_output_frequency(f_out)
b = np.array(lmx.get_register_dump())

print(lmx.OUTA_PWR.value)

Solutions:
  i    f_vco   DIV MIN_N DLY_SEL    n    R   R_pre f_fpd Metric
----------------------------------------------------------------
  0: 9830.40 /   2    28       1   40    1     1  245.76 80
Choosing solution 0 with minimal metric 80
31


In [29]:
lmx = ipq_pynq_utils.LMX2594(245.76)

lmx.set_output_frequency(f_out)
b = np.array(lmx.get_register_dump())
np.where(a[::-1] != b[::-1])

Solutions:
  i    f_vco   DIV MIN_N DLY_SEL    n    R   R_pre f_fpd Metric
----------------------------------------------------------------
  0: 9830.40 /   2    28       1   40    1     1  245.76 80
Choosing solution 0 with minimal metric 80


(array([  0,  14,  17,  60,  72,  78,  79,  80,  82,  84,  85,  97,  98,
        101, 102, 105, 106]),)

In [30]:
for f in lmx_clock_files:
    lmx.init_from_file(f"../ipq_pynq_utils/data/clockFiles/{f}")
    lmx.set_output_frequency(f_out)
    b = np.array(lmx.get_register_dump())
    print(f, np.where(a[::-1] != b[::-1]))

Solutions:
  i    f_vco   DIV MIN_N DLY_SEL    n    R   R_pre f_fpd Metric
----------------------------------------------------------------
  0: 9830.40 /   2    28       1   40    1     1  245.76 80
Choosing solution 0 with minimal metric 80
LMX2594_REF-245M76__OUT-0204M80_10172019_I_D.txt (array([], dtype=int64),)
Solutions:
  i    f_vco   DIV MIN_N DLY_SEL    n    R   R_pre f_fpd Metric
----------------------------------------------------------------
  0: 9830.40 /   2    28       1   40    1     1  245.76 80
Choosing solution 0 with minimal metric 80
LMX2594_REF-245M76__OUT-0245M76_10172019_I_D.txt (array([], dtype=int64),)
Solutions:
  i    f_vco   DIV MIN_N DLY_SEL    n    R   R_pre f_fpd Metric
----------------------------------------------------------------
  0: 9830.40 /   2    28       1   40    1     1  245.76 80
Choosing solution 0 with minimal metric 80
LMX2594_REF-245M76__OUT-0491M52_10172019_I_D.txt (array([], dtype=int64),)
Solutions:
  i    f_vco   DIV MIN_N DLY_SEL   

In [38]:
for addr,v in lmx.get_register_dump(with_addr=True):
    print(f"R{addr}\t0x{v:06X}")

R109	0x6D0000
R108	0x6C0000
R107	0x6B0000
R106	0x6A0000
R105	0x690021
R104	0x680000
R103	0x670000
R102	0x663F80
R101	0x650011
R100	0x640000
R99	0x630000
R98	0x620200
R97	0x610888
R96	0x600000
R95	0x5F0000
R94	0x5E0000
R93	0x5D0000
R92	0x5C0000
R91	0x5B0000
R90	0x5A0000
R89	0x590000
R88	0x580000
R87	0x570000
R86	0x560000
R85	0x55D300
R84	0x540001
R83	0x530000
R82	0x521E00
R81	0x510000
R80	0x506666
R79	0x4F0026
R78	0x4E0003
R77	0x4D0000
R76	0x4C000C
R75	0x4B0840
R74	0x4A0000
R73	0x49003F
R72	0x480001
R71	0x470081
R70	0x46C350
R69	0x450000
R68	0x4403E8
R67	0x430000
R66	0x4201F4
R65	0x410000
R64	0x401388
R63	0x3F0000
R62	0x3E0322
R61	0x3D00A8
R60	0x3C0000
R59	0x3B0001
R58	0x3A8001
R57	0x390020
R56	0x380000
R55	0x370000
R54	0x360000
R53	0x350000
R52	0x340820
R51	0x330080
R50	0x320000
R49	0x314180
R48	0x300300
R47	0x2F0300
R46	0x2E07FC
R45	0x2DC0C0
R44	0x2C1F80
R43	0x2B0000
R42	0x2A0000
R41	0x290000
R40	0x280000
R39	0x270000
R38	0x260000
R37	0x250104
R36	0x240C35
R35	0x230004
R34	0x220000
R3